## Part 1: Preprocessing

In [26]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [27]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [28]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]



In [29]:
# Create a list of at least 10 column names to use as X data
X_columns = ["Education", "Age", "DistanceFromHome", "JobSatisfaction", "OverTime", "StockOptionLevel", "WorkLifeBalance", "YearsAtCompany", "YearsSinceLastPromotion", "NumCompaniesWorked"]



# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [30]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [31]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train["OverTime"] = X_train["OverTime"].map({"Yes": 1, "No": 0})
X_test["OverTime"] = X_test["OverTime"].map({"Yes": 1, "No": 0})

In [32]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [33]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
department_encoder = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[["Department"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = department_encoder.transform(y_train[["Department"]])
y_test_department = department_encoder.transform(y_test[["Department"]])


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [34]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[["Attrition"]])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = attrition_encoder.transform(y_train[["Attrition"]])
y_test_attrition = attrition_encoder.transform(y_test[["Attrition"]])


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Create, Compile, and Train the Model

In [35]:
# Find the number of columns in the X training data
num_features = X_train_scaled.shape[1]

# Create the input layer
inputs = layers.Input(shape=(num_features,))

# Create at least two shared layers
shared1 = layers.Dense(64, activation="relu")(inputs)
shared2 = layers.Dense(128, activation="relu")(shared1)

In [36]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation="relu")(shared2)

# Create the output layer
department_output = layers.Dense(3, activation="softmax", name="department_output")(department_hidden)


In [37]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation="relu")(shared2)

# Create the output layer
attrition_output = layers.Dense(2, activation="softmax", name="attrition_output")(attrition_hidden)


In [38]:
# Create the model
model = Model(inputs=inputs, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 64)                   704       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 128)                  8320      ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 32)                   4128      ['dense_1[0][0]']             
                                                                                              

In [39]:
# Train the model
history = model.fit(
    X_train_scaled,
    {"department_output": y_train_department, "attrition_output": y_train_attrition},
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


Epoch 1/100
34/34 [==============================] - 5s 38ms/step - loss: 1.4455 - department_output_loss: 0.9226 - attrition_output_loss: 0.5229 - department_output_accuracy: 0.5851 - attrition_output_accuracy: 0.7675 - val_loss: 1.2977 - val_department_output_loss: 0.8433 - val_attrition_output_loss: 0.4544 - val_department_output_accuracy: 0.6356 - val_attrition_output_accuracy: 0.7966
Epoch 2/100
34/34 [==============================] - 1s 17ms/step - loss: 1.1946 - department_output_loss: 0.7761 - attrition_output_loss: 0.4185 - department_output_accuracy: 0.6541 - attrition_output_accuracy: 0.8374 - val_loss: 1.2606 - val_department_output_loss: 0.8528 - val_attrition_output_loss: 0.4078 - val_department_output_accuracy: 0.6356 - val_attrition_output_accuracy: 0.7966
Epoch 3/100
34/34 [==============================] - 0s 6ms/step - loss: 1.1540 - department_output_loss: 0.7661 - attrition_output_loss: 0.3879 - department_output_accuracy: 0.6531 - attrition_output_accuracy: 0.843

In [40]:
# Evaluate the model with the testing data
model.evaluate(X_test_scaled, [y_test_department, y_test_attrition])

10/10 [==============================] - 0s 3ms/step - loss: 3.6777 - department_output_loss: 2.3889 - attrition_output_loss: 1.2888 - department_output_accuracy: 0.5816 - attrition_output_accuracy: 0.8401


[3.6777074337005615,
 2.3889033794403076,
 1.288804054260254,
 0.581632673740387,
 0.8401360511779785]

In [41]:
# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {model.evaluate(X_test_scaled, [y_test_department, y_test_attrition])[3]}")
print(f"Attrition predictions accuracy: {model.evaluate(X_test_scaled, [y_test_department, y_test_attrition])[4]}")

10/10 [==============================] - 0s 3ms/step - loss: 3.6777 - department_output_loss: 2.3889 - attrition_output_loss: 1.2888 - department_output_accuracy: 0.5816 - attrition_output_accuracy: 0.8401
Department predictions accuracy: 0.581632673740387
10/10 [==============================] - 0s 3ms/step - loss: 3.6777 - department_output_loss: 2.3889 - attrition_output_loss: 1.2888 - department_output_accuracy: 0.5816 - attrition_output_accuracy: 0.8401
Attrition predictions accuracy: 0.8401360511779785


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric for this data, especially for the Attrition prediction. The dataset appears imbalanced (788 "No" vs 314 "Yes" for Attrition), so metrics like precision, recall, or F1-score could provide a more nuanced view of model performance, particularly for the minority class.
2. The output layers use softmax activation functions, which is appropriate for multi-class classification (Department with 3 classes) and binary classification (Attrition with 2 classes). Softmax ensures the outputs sum to 1 and can be interpreted as probabilities.
3. Ways to improve the model:

 - Address class imbalance (e.g. oversampling, undersampling, or using class weights)
 - Feature engineering or selection to focus on most relevant predictors
 - Trying different architectures (e.g. more/fewer layers, different neuron counts)
 - Using regularization techniques to prevent overfitting
 - Experimenting with different optimization algorithms or learning rates
 - Collecting more data if possible